In [103]:
import xml.etree.ElementTree as ET
import numpy as np
from copy import deepcopy

In [2]:
tree = ET.parse('Składnica-frazowa-171220/NKJP_1M_0401000002/morph_2-p/morph_2.52-s.xml')
tree

In [230]:
tree = ET.parse('Składnica-frazowa-171220/NKJP_1M_0401000002/morph_4-p/morph_4.67-s.xml')
tree

In [231]:
def check_sentence(forest):
    
    """
    Funkcja sprawdza poprawnosc wypowiedzenia - czy istnieje dla niego poprawne drzewo - 
    wypowiedzenie jest poprawne jesli base_answer na polu "type" ma wartosc "FULL".
    Zwraca krotke dwuelementowa:
    * wartosc logiczna, mowiaca o tym, czy jest drzewo poprawne
    * wartosc pola "type" w wezle base_answer
    
    forest - las drzew [xml.etree.ElementTree.ElementTree]
    """
    
    base_answer_type = forest.getroot().find('.//answer-data//base-answer').attrib["type"]
    correct = base_answer_type == "FULL"

    if not correct:
        raise AssertionError("Sentence is not correct: Node <base-answer> has type value " + base_answer_type  + " instead of 'FULL'")
        
    pass


def get_random_tree(forest, random_state=None):
    
    """
    Funkcja zwraca losowe drzewo z upakowanego lasu (forest).
    Dla lasu, w ktorym nie ma poprawnego drzewa funkcja wyrzuca blad.
    
    forest - las drzew [xml.etree.ElementTree.ElementTree]
    """

    # sprawdzenie poprawnosci lasu i ewentualne wypisanie
    check_sentence(forest)
    
    # ustawiamy ziarno
    if random_state is not None:
        np.random.seed(random_state)
            
            
    root_old = forest.getroot()
    root_new = ET.Element("tree",root_old.attrib)
    
    
    # las sklada sie z drzew (wezly "node") oraz dodatkowych danych (inne wezly) -
    # tresc wypowiedzenia, statystyki lasu, itd. - i tutaj przepisujemy te wezly
    features = root_old.getchildren()
    for feature in features:
        if feature.tag != "node": 
            feature_copy = deepcopy(feature)
            if feature_copy.tag == "stats":
                feature_copy.tag = "forest-stats"
                
            root_new.append(feature_copy) # modyfikujemy tag wezla wiec potrzebna kopia, zeby nie zmodyfikowac oryginalnego drzewa
            
    # definiujemy rekurencyjna funkcje, ktora bedzie przechodzic po lesie i
    # kolekcjonowac wezly, tworzac losowe drzewo.
    # drzewo jest tworzone na korzeniu root_new.
    def add_random_children(current_node_old):
        
        current_node_new = ET.SubElement(root_new, current_node_old.tag, current_node_old.attrib)
        
        features = current_node_old.getchildren()
        # kazdy "node" jest terminalem albo nieterminalem i ma opis wlasnosci
        # i tutaj wyciagamy te wlasnosci z wezla innego niz "children"
        for feature in features:
            if feature.tag != "children": 
                current_node_new.append(feature)
        
        children_old = current_node_old.findall("children")
        if len(children_old) == 0: #jestesmy w lisciu wiec konczymy dzialanie funkcji
            return None
        random_children_old = children_old[np.random.choice(len(children_old),1)[0]]
        random_children_new = ET.SubElement(current_node_new, random_children_old.tag, random_children_old.attrib)
        for child_old in random_children_old.getchildren():
            x = ET.SubElement(random_children_new, child_old.tag, child_old.attrib)
            next_node = root_old.find('.//node[@nid="' + x.attrib["nid"] + '"]')
            add_random_children(next_node)
        
    
    # wezel startowy (przyjmujemy, ze node z id=0 jest zawsze pierwszy):
    node_0 = root_old.find('.//node[@nid="0"]') 
    
    # konstruujemy drzewo:
    add_random_children(node_0)
    
    
    return ET.ElementTree(root_new)
       

In [232]:
random_tree = get_random_tree(tree)

In [225]:
random_tree.write("test.xml")

In [233]:
ET.dump(random_tree)

<tree grammar_no="1505562921" sent_id="NKJP_1M_0401000002/morph_4-p/morph_4.67-s"><text>Papież Formosus zmarł w kwietniu 896 w wyniku nieznanej choroby lub otrucia.</text>
  <startnode from="0" to="13">wypowiedzenie</startnode>
  <forest-statsxxxxxxxxxxxx cputime="10.487813580000001" inferences="7448127" nodes="124" trees="173" />
    <answer-data>
        <base-answer type="FULL" username="none">
            <comment>AUTO</comment>
        </base-answer>
        <extra-answer type="FULL" username="witoldk">
            <comment>AUTO</comment>
        </extra-answer>
        <extra-answer type="FULL" username="piotrb">
            <comment>AUTO</comment>
        </extra-answer>
    </answer-data>
  <node chosen="true" from="0" nid="0" subtrees="173" to="13"><nonterminal>
      <category>wypowiedzenie</category>
    </nonterminal>
    <children chosen="true" rule="w"><child from="0" head="true" nid="1" to="12" /><child from="12" head="false" nid="71" to="13" /></children></node><node ch

In [121]:
tree = random_tree

In [213]:
t

In [212]:
random_tree = t.getroot()

In [202]:
def tree_height(tree, node_id=0):
    
    """
    Funkcja oblicza wysokosc drewa (dlugosc najdluzszej sciezki od korzenia do liscia)
    
    tree - drzewo lub korzen drzewa
    """
    if type(tree)==ET.Element:
        node = tree
    else:
        node = tree.getroot()
    node = node.find('.//node[@nid="' + str(node_id) + '"]')
    children = node.findall(".//children//child")
    
    if len(children)==0:
        return 1
    else:
        children_nodes = [child.attrib["nid"] for child in children]
        return 1+max([tree_height(tree,x) for x in children_nodes])
        

In [204]:
tree_height(t)

10

In [205]:
def number_of_nodes(tree):
    """
    Zwraca liczbe wezlow w drzewie.
    
    tree - drzewo lub korzen drzewa
    """
    if type(tree)==ET.Element:
        return len(tree.findall("node"))
    else:
        return len(tree.getroot().findall("node")) 

In [215]:
number_of_nodes(t)

49

In [151]:
ET.dump(t)

<forest grammar_no="1505562921" sent_id="NKJP_1M_0401000002/morph_4-p/morph_4.67-s"><text>Papież Formosus zmarł w kwietniu 896 w wyniku nieznanej choroby lub otrucia.</text>
  <startnode from="0" to="13">wypowiedzenie</startnode>
  <forest_stats cputime="10.487813580000001" inferences="7448127" nodes="124" trees="173" />
    <answer-data>
        <base-answer type="FULL" username="none">
            <comment>AUTO</comment>
        </base-answer>
        <extra-answer type="FULL" username="witoldk">
            <comment>AUTO</comment>
        </extra-answer>
        <extra-answer type="FULL" username="piotrb">
            <comment>AUTO</comment>
        </extra-answer>
    </answer-data>
  <node chosen="true" from="0" nid="0" subtrees="173" to="13"><nonterminal>
      <category>wypowiedzenie</category>
    </nonterminal>
    <children chosen="true" rule="w"><child from="0" head="true" nid="1" to="12" /><child from="12" head="false" nid="71" to="13" /></children></node><node chosen="true

In [149]:
random_tree

<Element 'forest' at 0x7f66dae6e408>

In [150]:
tree_height(random_tree)

4